<a href="https://colab.research.google.com/github/akikoiwamizu/enron-vizards/blob/main/data/Enron_data_cleansing_collab_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Downloading the file from Kaggle requires you to go to the account tab of the My Profile 
# section and click on Create New API Token. This will download a kaggle.json file.

## ! mkdir /root/.kaggle/
## ! cp kaggle.json /root/.kaggle/
## ! chmod 600 ~/.kaggle/kaggle.json

# Once you have the kaggle.json file download, move it to this location on your local machine
# to access the Kaggle API. For me, the following command did the trick:

##! cp ~/Downloads/kaggle.json  ~/.kaggle/
##! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# This will fail if the files are already on your local machine in this directory.

##! kaggle datasets download wcukierski/enron-email-dataset
##! unzip enron-email-dataset.zip 

In [ ]:
# Remove zip file after csv successfully obtained.

##! rm enron-email-dataset.zip 

# Enron data cleansing
## Import dataset

In [1]:
import pandas as pd
import numpy as np
import os
from email.parser import Parser
from datetime import datetime
import re

In [2]:
df = pd.read_csv('emails.csv')
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [3]:
emails_lst = []

for i in range(len(df)):
    email = Parser().parsestr(df.message[i])
    emails_lst.append(dict(zip(email.keys(), email.values())))
    emails_lst[-1]["Body"] = email.get_payload()

In [4]:
emails = pd.DataFrame(emails_lst)
emails.head()

,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,...,X-bcc,X-Folder,X-Origin,X-FileName,Body,Cc,Bcc,Time,Attendees,Re
0,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,...,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,NaN,NaN,NaN,NaN,NaN
1,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,...,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,NaN,NaN,NaN,NaN,NaN
2,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,...,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,NaN,NaN,NaN,NaN,NaN
3,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,...,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",NaN,NaN,NaN,NaN,NaN
4,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,...,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,NaN,NaN,NaN,NaN,NaN


We just want to keep the useful stuff and drop all unnecessary columns.

In [5]:
emails.drop(['Message-ID','Mime-Version','Content-Type','Content-Transfer-Encoding','X-FileName'], axis=1, inplace=True)
emails.head()

,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,Body,Cc,Bcc,Time,Attendees,Re
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,Here is our forecast\n\n,NaN,NaN,NaN,NaN,NaN
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,Traveling to have a business meeting takes the...,NaN,NaN,NaN,NaN,NaN
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,test successful. way to go!!!,NaN,NaN,NaN,NaN,NaN
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,"Randy,\n\n Can you send me a schedule of the s...",NaN,NaN,NaN,NaN,NaN
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,Let's shoot for Tuesday at 11:45.,NaN,NaN,NaN,NaN,NaN


Now let's have a look on the data.

In [6]:
display(emails.describe())
emails.isna().sum()

,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,Body,Cc,Bcc,Time,Attendees,Re
count,517401,517401,495554,517401,517372,517372,517372,517372,517372,517372,517401,127881,127881,1,2,2
unique,224128,20328,58563,159290,27980,73552,33701,132,5335,259,249025,29595,29595,1,1,1
top,"Wed, 27 Jun 2001 16:02:00 -0700 (PDT)",kay.mann@enron.com,pete.davis@enron.com,,Kay Mann,,,,\Kay_Mann_June2001_1\Notes Folders\All documents,Kaminski-V,"As you know, Enron Net Works (ENW) and Enron G...","albert.meyers@enron.com, bill.williams@enron.c...","albert.meyers@enron.com, bill.williams@enron.c...",9:00am -\n 10:00am,"Martin Chavez, CEO & Founder of Kiodex, Raj\n ...",Fitzgertald project Kiodex
freq,1118,16735,9155,19187,16324,9124,388486,517197,6639,26995,112,3106,3106,1,2,2


Date              0
From              0
To            21847
Subject           0
X-From           29
X-To             29
X-cc             29
X-bcc            29
X-Folder         29
X-Origin         29
Body              0
Cc           389520
Bcc          389520
Time         517400
Attendees    517399
Re           517399
dtype: int64

There seem to be a couple of NaNs for the authors and recicpients of some emails which makes them useless for our analysis. We hence drop them.

In [7]:
emails.dropna(subset=["X-From","X-To"], how="all", inplace=True)
emails.reset_index(drop=True,inplace=True)
emails.isna().sum()

Date              0
From              0
To            21825
Subject           0
X-From            0
X-To              0
X-cc              0
X-bcc             0
X-Folder          0
X-Origin          0
Body              0
Cc           389491
Bcc          389491
Time         517372
Attendees    517372
Re           517370
dtype: int64

Let's try to get the date into a nice format.

In [8]:
emails = pd.concat((emails, pd.DataFrame(np.reshape([y for x in emails.Date.apply(lambda x: x.split()[1:6]) for y in x],
                                                (len(emails),5)), 
             columns=["day","month","year","time","tzdiff"])), axis=1)
emails.head()

,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,...,Cc,Bcc,Time,Attendees,Re,day,month,year,time,tzdiff
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,...,NaN,NaN,NaN,NaN,NaN,14,May,2001,16:39:00,-0700
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,...,NaN,NaN,NaN,NaN,NaN,4,May,2001,13:51:00,-0700
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,...,NaN,NaN,NaN,NaN,NaN,18,Oct,2000,03:00:00,-0700
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,...,NaN,NaN,NaN,NaN,NaN,23,Oct,2000,06:13:00,-0700
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,...,NaN,NaN,NaN,NaN,NaN,31,Aug,2000,05:07:00,-0700


In [9]:
#Convert to int
emails.day = emails.day.astype(int)
emails.year = emails.year.astype(int)
#Convert month name to number
d = dict(zip(pd.date_range('2000-01-01', freq='M', periods=12).strftime('%b'),range(1,13)))
emails.month = emails.month.map(d)

The Enron scandal ended with its bankrupcy in Dec 2001. Hence, data before this date should be irrelevant or non-existent. But we could also go for 2007 when the company completely ceased to exist. Let's check:

In [10]:
display(emails[emails.year > 2001])
display(emails[emails.year > 2007])

,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,...,Cc,Bcc,Time,Attendees,Re,day,month,year,time,tzdiff
2013,"Tue, 1 Jan 2002 14:34:36 -0800 (PST)",subscriptions@intelligencepress.com,pallen@enron.com,"NGI Publications - Tuesday, January 1st 2002",subscriptions@intelligencepress.com@ENRON,pallen@enron.com,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\Inbox",Allen-P,...,NaN,NaN,NaN,NaN,NaN,1,1,2002,14:34:36,-0800
2014,"Tue, 1 Jan 2002 14:46:05 -0800 (PST)",subscriptions@intelligencepress.com,pallen@enron.com,"NGI Publications - Wednesday, January 2nd 2002",subscriptions@intelligencepress.com@ENRON,pallen@enron.com,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\Inbox",Allen-P,...,NaN,NaN,NaN,NaN,NaN,1,1,2002,14:46:05,-0800
2015,"Tue, 1 Jan 2002 17:19:40 -0800 (PST)",arsystem@mailman.enron.com,k..allen@enron.com,Your Approval is Overdue: Access Request for m...,ARSystem <ARSystem@mailman.enron.com>@ENRON,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...",,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\Inbox",Allen-P,...,NaN,NaN,NaN,NaN,NaN,1,1,2002,17:19:40,-0800
6111,"Tue, 1 Jan 2002 11:04:18 -0800 (PST)",msagel@home.com,jarnold@enron.com,Market update,"""Mark Sagel"" <msagel@home.com>@ENRON",John Arnold <jarnold@enron.com>,Mike Maggi <mmaggi@enron.com>,,"\John_Arnold_Jan2002_1\Arnold, John\Inbox",Arnold-J,...,mmaggi@enron.com,mmaggi@enron.com,NaN,NaN,NaN,1,1,2002,11:04:18,-0800
8264,"Tue, 1 Jan 2002 01:11:25 -0800 (PST)",arsystem@mailman.enron.com,harry.arora@enron.com,Your Approval is Overdue: Access Request for j...,ARSystem <ARSystem@mailman.enron.com>,"Arora, Harry </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...",,,"\Harry_Arora_Jan2002\Arora, Harry\Inbox",Arora-H,...,NaN,NaN,NaN,NaN,NaN,1,1,2002,01:11:25,-0800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517341,"Mon, 7 Jan 2002 13:04:59 -0800 (PST)",john.zufferli@enron.com,frank.hayden@enron.com,RE:,"Zufferli, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...","Hayden, Frank </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",,,"\ExMerge - Zufferli, John\Sent Items",ZUFFERLI-J,...,NaN,NaN,NaN,NaN,NaN,7,1,2002,13:04:59,-0800
517342,"Mon, 7 Jan 2002 13:01:08 -0800 (PST)",john.zufferli@enron.com,livia_zufferli@monitor.com,RE:,"Zufferli, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...",'Livia_Zufferli@Monitor.com@ENRON','jzufferli@shaw.ca',,"\ExMerge - Zufferli, John\Sent Items",ZUFFERLI-J,...,jzufferli@shaw.ca,jzufferli@shaw.ca,NaN,NaN,NaN,7,1,2002,13:01:08,-0800
517343,"Wed, 2 Jan 2002 09:55:40 -0800 (PST)",john.zufferli@enron.com,frank.senders@pandora.be,Training,"Zufferli, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...",'frank.senders@pandora.be',,,"\ExMerge - Zufferli, John\Sent Items",ZUFFERLI-J,...,NaN,NaN,NaN,NaN,NaN,2,1,2002,09:55:40,-0800
517350,"Tue, 5 Feb 2002 15:18:24 -0800 (PST)",john.zufferli@enron.com,janet.mulero@ubsw.com,RE: Market Risk Meeting Today - John Zufferli,"Zufferli, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...",'Janet.Mulero@ubsw.com@ENRON',,,"\ExMerge - Zufferli, John\Sent Items",ZUFFERLI-J,...,NaN,NaN,NaN,NaN,NaN,5,2,2002,15:18:24,-0800


,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,...,Cc,Bcc,Time,Attendees,Re,day,month,year,time,tzdiff
43805,"Thu, 24 Dec 2020 15:36:06 -0800 (PST)",jody421@hotmail.com,undisclosed.recipients@mailman.enron.com,Tired Of Searching For Love In All The Wrong P...,JODY421@hotmail.com,Undisclosed.Recipients@mailman.enron.com,,,"\Larry_Campbell_Jan2002_2\Campbell, Larry\Inbox",Campbell-L,...,NaN,NaN,NaN,NaN,NaN,24,12,2020,15:36:06,-0800
43848,"Tue, 29 Dec 2020 12:53:46 -0800 (PST)",jody421@hotmail.com,undisclosed.recipients@mailman.enron.com,Jennifer Lopez - Nudity,JODY421@hotmail.com,Undisclosed.Recipients@mailman.enron.com,,,"\Larry_Campbell_Jan2002_2\Campbell, Larry\Inbox",Campbell-L,...,NaN,NaN,NaN,NaN,NaN,29,12,2020,12:53:46,-0800
84577,"Tue, 27 Nov 2012 20:47:24 -0800 (PST)",dvdnow_77395@xoip.com,NaN,Copy Your Favorite DVD Movies ... Great Gift I...,dvdnow_77395@xoip.com,,,,"\DDAVIS (Non-Privileged)\Davis, Dana\Deleted I...",Davis-D,...,NaN,NaN,NaN,NaN,NaN,27,11,2012,20:47:24,-0800
84582,"Sun, 18 Nov 2012 08:28:05 -0800 (PST)",dvdnow_52772@xoip.com,NaN,Copy Your Favorite DVD Movies !!!,dvdnow_52772@xoip.com,,,,"\DDAVIS (Non-Privileged)\Davis, Dana\Deleted I...",Davis-D,...,NaN,NaN,NaN,NaN,NaN,18,11,2012,08:28:05,-0800
508347,"Sun, 26 May 2024 03:49:57 -0700 (PDT)",pse6yl706@aloha.net,NaN,(None),PSE6Yl706@aloha.net@ENRON <IMCEANOTES-PSE6Yl70...,undisclosed-recipients:;@ENRON <??Sundisclosed...,,,"\Williams, Jason (Trading)\Williams, Jason (Tr...",WILLIAMS-J,...,NaN,NaN,NaN,NaN,NaN,26,5,2024,03:49:57,-0700
517010,"Mon, 4 Jan 2044 14:48:58 -0800 (PST)",cramer@cadvision.com,john.zufferli@enron.com,trades,"""Erik Cramer"" <cramer@cadvision.com>@ENRON","Zufferli, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...",,,"\ExMerge - Zufferli, John\Inbox\Quotes",ZUFFERLI-J,...,NaN,NaN,NaN,NaN,NaN,4,1,2044,14:48:58,-0800
517011,"Mon, 4 Jan 2044 08:59:46 -0800 (PST)",cramer@cadvision.com,john.zufferli@enron.com,Trades,"""Erik Cramer"" <cramer@cadvision.com>@ENRON","Zufferli, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...",,,"\ExMerge - Zufferli, John\Inbox\Quotes",ZUFFERLI-J,...,NaN,NaN,NaN,NaN,NaN,4,1,2044,08:59:46,-0800
517013,"Sat, 2 Jan 2044 15:46:00 -0800 (PST)",cramer@cadvision.com,john.zufferli@enron.com,trades jan 2002,"""Erik Cramer"" <cramer@cadvision.com>@ENRON","Zufferli, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...",,,"\ExMerge - Zufferli, John\Inbox\Quotes",ZUFFERLI-J,...,NaN,NaN,NaN,NaN,NaN,2,1,2044,15:46:00,-0800
517016,"Mon, 28 Dec 2043 11:34:12 -0800 (PST)",cramer@cadvision.com,"linsider.jed@enron.com, john.zufferli@enron.co...",marks.xls,"""Erik Cramer"" <cramer@cadvision.com>@ENRON","Jed Linsider <jed.linsider@mirant.com>, Zuffer...",,,"\ExMerge - Zufferli, John\Inbox\Quotes",ZUFFERLI-J,...,NaN,NaN,NaN,NaN,NaN,28,12,2043,11:34:12,-0800


There is some erroneous data with years in the future. Let's stick to the 2007 threshold then.

In [11]:
emails = emails[emails.year <= 2007]
emails.head()

,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,...,Cc,Bcc,Time,Attendees,Re,day,month,year,time,tzdiff
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,...,NaN,NaN,NaN,NaN,NaN,14,5,2001,16:39:00,-0700
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,...,NaN,NaN,NaN,NaN,NaN,4,5,2001,13:51:00,-0700
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,...,NaN,NaN,NaN,NaN,NaN,18,10,2000,03:00:00,-0700
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,...,NaN,NaN,NaN,NaN,NaN,23,10,2000,06:13:00,-0700
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,...,NaN,NaN,NaN,NaN,NaN,31,8,2000,05:07:00,-0700


What about minimum values? Enron was founded in 1985 so that seems to be a reasonable threshold. However, Enron was founded by a merger of two companies and one of them was founded in 1931. But emails were invented and used in companies from 1975 on. Let's use this a threshold instead.

In [12]:
emails[emails.year < 1975]

,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,...,Cc,Bcc,Time,Attendees,Re,day,month,year,time,tzdiff
8110,"Wed, 21 Dec 0001 22:31:43 -0800 (PST)",enron_update@concureworkplace.com,harora@enron.com,<<Concur Expense Document>> - East Power Lunch...,enron_update@concureworkplace.com,Harpreet Arora <harora@enron.com>,,,"\Harry_Arora_Jan2002\Arora, Harry\Deleted Items",Arora-H,...,NaN,NaN,NaN,NaN,NaN,21,12,1,22:31:43,-0800
8130,"Sat, 24 Dec 0001 22:26:44 -0800 (PST)",enron_update@concureworkplace.com,harora@enron.com,<<Concur Expense Document>> - East Power Lunch...,enron_update@concureworkplace.com,Harpreet Arora <harora@enron.com>,,,"\Harry_Arora_Jan2002\Arora, Harry\Deleted Items",Arora-H,...,NaN,NaN,NaN,NaN,NaN,24,12,1,22:26:44,-0800
32820,"Sat, 6 Aug 0001 00:06:06 -0800 (PST)",payables.ibuyit@enron.com,robert.benson@enron.com,Action Requested: Past Due Invoice,iBuyit Payables </O=ENRON/OU=NA/CN=RECIPIENTS/...,"Benson, Robert </O=ENRON/OU=NA/CN=RECIPIENTS/C...",,,"\RBENSON (Non-Privileged)\Benson, Robert\Inbox",Benson-R,...,NaN,NaN,NaN,NaN,NaN,6,8,1,00:06:06,-0800
32838,"Sat, 13 Aug 0001 00:11:21 -0800 (PST)",payables.ibuyit@enron.com,robert.benson@enron.com,Action Requested: Past Due Invoice,iBuyit Payables </O=ENRON/OU=NA/CN=RECIPIENTS/...,"Benson, Robert </O=ENRON/OU=NA/CN=RECIPIENTS/C...",,,"\RBENSON (Non-Privileged)\Benson, Robert\Inbox",Benson-R,...,NaN,NaN,NaN,NaN,NaN,13,8,1,00:11:21,-0800
32903,"Sun, 28 Aug 0001 00:08:57 -0800 (PST)",ipayit@enron.com,robert.benson@enron.com,Action Requested: Past Due Invoice,iPayit </O=ENRON/OU=NA/CN=RECIPIENTS/CN=MBX_IP...,"Benson, Robert </O=ENRON/OU=NA/CN=RECIPIENTS/C...",,,"\RBENSON (Non-Privileged)\Benson, Robert\Inbox",Benson-R,...,NaN,NaN,NaN,NaN,NaN,28,8,1,00:08:57,-0800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507310,"Sat, 3 Dec 0001 09:45:56 -0800 (PST)",enron_update@concureworkplace.com,mark.whitt@enron.com,<<Concur Expense Document>> - Nov/Dec bills,enron_update@concureworkplace.com,"Whitt, Mark </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",,,"\Mark_Whitt_Jan2002_1\Whitt, Mark\Inbox",Whitt-M,...,NaN,NaN,NaN,NaN,NaN,3,12,1,09:45:56,-0800
507358,"Sun, 4 Dec 0001 03:49:44 -0800 (PST)",enron_update@concureworkplace.com,mark.whitt@enron.com,<<Concur Expense Document>> - MW112801,enron_update@concureworkplace.com,"Whitt, Mark </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",,,"\Mark_Whitt_Jan2002_1\Whitt, Mark\Inbox",Whitt-M,...,NaN,NaN,NaN,NaN,NaN,4,12,1,03:49:44,-0800
507395,"Mon, 21 Nov 0001 09:44:22 -0800 (PST)",enron_update@concureworkplace.com,mark.whitt@enron.com,<<Concur Expense Document>> - PL112101,enron_update@concureworkplace.com,"Whitt, Mark </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",,,"\MWHITT (Non-Privileged)\Whitt, Mark\Inbox",Whitt-M,...,NaN,NaN,NaN,NaN,NaN,21,11,1,09:44:22,-0800
507418,"Tue, 6 Dec 0001 15:40:44 -0800 (PST)",enron_update@concureworkplace.com,mark.whitt@enron.com,<<Concur Expense Document>> - 121001,enron_update@concureworkplace.com,"Whitt, Mark </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",,,"\Mark_Whitt_Jan2002_1\Whitt, Mark\Inbox",Whitt-M,...,NaN,NaN,NaN,NaN,NaN,6,12,1,15:40:44,-0800


Seems like we will drop junk data. Let's do that.

In [13]:
emails = emails[emails.year >= 1975]
emails.head()

,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,...,Cc,Bcc,Time,Attendees,Re,day,month,year,time,tzdiff
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,...,NaN,NaN,NaN,NaN,NaN,14,5,2001,16:39:00,-0700
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,...,NaN,NaN,NaN,NaN,NaN,4,5,2001,13:51:00,-0700
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,...,NaN,NaN,NaN,NaN,NaN,18,10,2000,03:00:00,-0700
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,...,NaN,NaN,NaN,NaN,NaN,23,10,2000,06:13:00,-0700
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,...,NaN,NaN,NaN,NaN,NaN,31,8,2000,05:07:00,-0700


Ok, now let's take a look on the names of people.

In [14]:
people = set().union(*[emails["X-To"],emails["X-From"]])
people

{'',
 "'tajmajulie@aol.com'",
 'Lindberg, Lorraine </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Llindbe>, McConnell, Mark </O=ENRON/OU=NA/CN=RECIPIENTS/CN=MMcConne>',
 'Watson, Kimberly </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Kwatson>, McConnell, Mark </O=ENRON/OU=NA/CN=RECIPIENTS/CN=MMcConne>, Kilmer III, Robert </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Rkilmer>, Lokey, Teb </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Tlokey>',
 'Smith, Jeff E. </O=ENRON/OU=NA/CN=RECIPIENTS/CN=JSMITH>',
 'Channing Strother <cstrother@MSHPC.COM>',
 'pyoung@eei.org',
 'Vernon, Clayton </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Cvernon>',
 'Ban Sharma',
 'hostmaster@register.com@ENRON',
 'Frausto, Ruthann </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Rfraust>',
 'Kean, Steven J. </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Skean>, Fleming, Rosalee </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Rflemin>, Fastow, Andrew </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Afastow>, Koenig, Mark </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Mkoenig>',
 'Alan B Aronowitz, Leslie Hansen, Stacy E Dickson, Harry M Collins',
 'Cook

There are some issues with the names. Let's fix them.

In [15]:
#Let's get rid of everything after "<"
emails["X-To"] = emails["X-To"].apply(lambda x: x[:x.find("<")-1] if x.find("<") > 0 else x)
emails["X-From"] = emails["X-From"].apply(lambda x: x[:x.find("<")-1] if x.find("<") > 0 else x)
emails["X-cc"] = emails["X-cc"].apply(lambda x: x[:x.find("<")-1] if x.find("<") > 0 else x)
emails["X-bcc"] = emails["X-bcc"].apply(lambda x: x[:x.find("<")-1] if x.find("<") > 0 else x)
emails[["X-To","X-From","X-cc","X-bcc"]].head()

,X-To,X-From,X-cc,X-bcc
0,Tim Belden,Phillip K Allen,,
1,John J Lavorato,Phillip K Allen,,
2,Leah Van Arsdall,Phillip K Allen,,
3,Randall L Gay,Phillip K Allen,,
4,Greg Piper,Phillip K Allen,,


In [16]:
annoying_lst = emails["X-To"][emails["X-To"].str.len().sort_values(ascending=False)[:10].index]
less_annoying_lst = emails["X-To"].sample(20)

In [17]:
def clean_name_format(names):
    #Variables
    emails_dict = {}
    emails_list = []
    name_drop = []
    remaining = []
    single = []
    multiple = []
    clean_names = []
    #One person with format "lastName, firstName optMiddleName"
    if names.strip(",").count(",") == 1:
        if len(names.split()) > 3:
            multiple = names.strip("\'").split(", ")
        else:
            single = names.strip("\'").split(", ")
            if len(single) == 1:
                single = single[0].strip("\'").split(",")
            single = single[1] + " " + single[0]
    else:
        #Separate string into list of strings
        names = names.strip(",").split(", ")
        #Separate email addresses from name and save in dictionary
        names = [x.replace('\\',"").replace('\"',"").strip(' ') for x in names]
        for name in names:
            #Assuming only one email per name
            email = re.findall(r'<[\w.+-]+@[\w-]+\.[\w.-]+>', name)
            if len(email) > 0:
                if len(name) > len(email[0]):
                    emails_dict[email[0]] = name.replace(email[0],"").replace("."," ").strip(" ").title()
                else:
                    emails_list.append(email[0].replace("."," ").title())
                name_drop.append(name)
        #Lookup names in email_dict
        for i, email in enumerate(emails_list):
            if email in emails_dict.values():
                emails_list[i] = emails_dict.get(i)    
        remaining = list(set(names) - set(name_drop))
        clean_names = list(set(emails_list + list(emails_dict.values()) + remaining))
    if single:
        clean_names.append(single)
    if multiple:
        clean_names = clean_names + multiple
    return clean_names

In [18]:
#Let's extract lists of recipients
emails["X-To"] = emails["X-To"].apply(lambda x: clean_name_format(x))
emails["X-From"] = emails["X-From"].apply(lambda x: clean_name_format(x))
emails["X-cc"] = emails["X-cc"].apply(lambda x: clean_name_format(x))
emails["X-bcc"] = emails["X-bcc"].apply(lambda x: clean_name_format(x))

In [19]:
emails.head()

,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,...,Cc,Bcc,Time,Attendees,Re,day,month,year,time,tzdiff
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,[Phillip K Allen],[Tim Belden],[],[],"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,...,NaN,NaN,NaN,NaN,NaN,14,5,2001,16:39:00,-0700
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,[Phillip K Allen],[John J Lavorato],[],[],"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,...,NaN,NaN,NaN,NaN,NaN,4,5,2001,13:51:00,-0700
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,[Phillip K Allen],[Leah Van Arsdall],[],[],\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,...,NaN,NaN,NaN,NaN,NaN,18,10,2000,03:00:00,-0700
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,,[Phillip K Allen],[Randall L Gay],[],[],\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,...,NaN,NaN,NaN,NaN,NaN,23,10,2000,06:13:00,-0700
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,[Phillip K Allen],[Greg Piper],[],[],\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,...,NaN,NaN,NaN,NaN,NaN,31,8,2000,05:07:00,-0700


Let's see if this looks better now.

In [ ]:
#people = set().union(*[emails["X-To"].apply(pd.Series).stack().reset_index(drop=True),
#              emails["X-From"].apply(pd.Series).stack().reset_index(drop=True),
#              emails["X-cc"].apply(pd.Series).stack().reset_index(drop=True),
#              emails["X-bcc"].apply(pd.Series).stack().reset_index(drop=True)])
#print(len(people))
#people

Looks a lot better now. The EDA will show if we have other issues with the data, but for now...

In [20]:
emails.to_csv("emails_clean.csv",index=False)